[2025/03/23 20:40:08] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/miza/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.3, det_db_unclip_ratio=2.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='CRNN', rec_model_dir='/home/miza/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, m

In [98]:
def match_boxes_within_distance(boxes, threshold=20):
    # List to store the groups of matched boxes
    matched_groups = []
    
    # Create a list to track the boxes already grouped
    grouped = [False] * len(boxes)

    # Loop through each box and compare ymin, ymax, and xmin
    for i in range(len(boxes)):
        if grouped[i]:  # Skip already grouped boxes
            continue
        
        # Start a new group for the current box
        group = [boxes[i]]
        label1 = boxes[i][1]  # Label of box 1
        box1 = boxes[i][0]  # Coordinates of box 1

        ymin1 = min([point[1] for point in box1])  # Find ymin for box 1
        ymax1 = max([point[1] for point in box1])  # Find ymax for box 1
        
        for j in range(i + 1, len(boxes)):
            if grouped[j]:  # Skip already grouped boxes
                continue
            box2 = boxes[j][0]  # Coordinates of box 2
            label2 = boxes[j][1]  # Label of box 2    
            ymin2 = min([point[1] for point in box2])  # Find ymin for box 2
            ymax2 = max([point[1] for point in box2])  # Find ymax for box 2
            
            if abs(ymin1 - ymin2) < threshold or abs(ymax1 - ymax2) < threshold:
                group.append(boxes[j])
                grouped[j] = True

        group.sort(key=lambda box: min([point[0] for point in box[0]]))  # Sort by xmin
        line = [box[1][0] for box in group]
        matched_groups.append(line)

    return matched_groups


In [ ]:
from paddleocr import PaddleOCR
import paddle

# Initialize PaddleOCR (English by default)
ocr = PaddleOCR(
    use_angle_cls=True,
    lang='pl',
    det_db_box_thresh=0.3,
    det_db_unclip_ratio=2.5,
    rec_algorithm='CRNN',
    rec_batch_num=6,
    use_space_char=True,
)  # need to specify use_angle_cls=True if you want to detect rotated text

# Path to your image
img_path = '/home/miza/Magisterka/src/data/images/1.jpg'

# Run OCR
result = ocr.ocr(
    img_path,
    cls=True,
)

matched_boxes = match_boxes_within_distance(result[0])

In [102]:
matched_boxes

[['SALATKAJARZY250g-D', '1szt.x4,504,500'],
 ['JOG ALE PITNY 290g-D', '1szt.x3.603.600'],
 ['KAJZERKA PREM 60g-D', '1szt.x0,490,490'],
 ['KAJZERKA PREM 60q-D', '1szt.x0,490,490']]